# Github Link : https://github.com/binalbariya/Data-Warehousing/blob/main/Assignment%204/Web%20Scraping.ipynb

# Here i will scrape data from an Ecommerce website and fetch name,price,rating and its about section

In [1]:
#import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.thewhiskyexchange.com"

In [3]:
#we will send a user-agent on every HTTP request, because if you make GET request using requests then 
#by default the user-agent is Python which might get blocked.

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}


In [4]:
productlinks = []
t={}
data=[]
c=0

# Firstly inspect the webpage using chrome devtools

In [5]:
# We need to make an HTTP call first. 
#Then we will extract the li element using BeautifulSoup.
k = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky').text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"product-grid__item"})
print(productlist)

[<li class="product-grid__item"><a class="product-card" href="/p/37325/suntory-torys-classic" onclick="_gaq.push(['_trackEvent', 'Products-GridView', 'click', '37325 : Suntory Torys Classic'])" title="Suntory Torys Classic"><div class="product-card__image-container"><img alt="Suntory Torys Classic" class="product-card__image lazy" data-original="https://img.thewhiskyexchange.com/480/japan_sun20.jpg" src="https://img.thewhiskyexchange.com/ph.png"/></div><div class="product-card__content"><p class="product-card__name"> Suntory Torys Classic </p><p class="product-card__meta"> 70cl / 37% </p></div><div class="product-card__data"><p class="product-card__price"> £30.45 </p><p class="product-card__unit-price"> (£43.50 per litre) </p></div></a></li>, <li class="product-grid__item"><a class="product-card" href="/p/45577/nikka-days" onclick="_gaq.push(['_trackEvent', 'Products-GridView', 'click', '45577 : Nikka Days'])" title="Nikka Days"><div class="product-card__image-container"><img alt="Nikk

# Next, we are getting the HTML for the items on this page. 
# inside each of these lists there is a link to the individual product page. We will scrape all those links from the productlist.

In [7]:
productlinks = []
for product in productlist:
        link = product.find("a",{"class":"product-card"}).get('href')                 
        productlinks.append(url + link)

In [13]:
#To do so we will introduce a for loop before making the HTTP call.
productlinks = []
for x in range(1,6):
    
    
    k = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky?pg={}&psize=24&sort=pasc'.format(x)).text  
    soup=BeautifulSoup(k,'html.parser')  
    productlist = soup.find_all("li",{"class":"product-grid__item"})

    for product in productlist:
        link = product.find("a",{"class":"product-card"}).get('href')
        productlinks.append(url + link)

In [14]:
print(productlinks)

['https://www.thewhiskyexchange.com/p/37325/suntory-torys-classic', 'https://www.thewhiskyexchange.com/p/45577/nikka-days', 'https://www.thewhiskyexchange.com/p/36362/suntory-toki', 'https://www.thewhiskyexchange.com/p/16917/akashi-blended-whisky', 'https://www.thewhiskyexchange.com/p/49001/tokinoka-white-blended-whisky', 'https://www.thewhiskyexchange.com/p/49821/hatozaki-blended-japanese-whisky', 'https://www.thewhiskyexchange.com/p/57521/suntory-toki-glass-pack', 'https://www.thewhiskyexchange.com/p/48272/mars-kasei-blended-whisky', 'https://www.thewhiskyexchange.com/p/2928/nikka-from-the-barrel', 'https://www.thewhiskyexchange.com/p/24587/togouchi-premium-blended-whisky', 'https://www.thewhiskyexchange.com/p/2935/nikka-pure-malt-red', 'https://www.thewhiskyexchange.com/p/49822/hatozaki-pure-malt-japanese-whisky', 'https://www.thewhiskyexchange.com/p/34970/tokinoka-black-blended-whisky', 'https://www.thewhiskyexchange.com/p/37317/suntory-chita-whisky', 'https://www.thewhiskyexchange

# The Name is under an h1 tag, the about text is under the div tag, price is under a p tag, and rating is under the span tag. Now, let's extract them.

In [15]:
data=[]
for link in productlinks:
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')

    try:
        price=hun.find("p",{"class":"product-action__price"}).text.replace('\n',"")
    except:
        price = None

    try:
        about=hun.find("div",{"class":"product-main__description"}).text.replace('\n',"")
    except:
        about=None

    try:
        rating = hun.find("div",{"class":"review-overview"}).text.replace('\n',"")
    except:
        rating=None

    try:
        name=hun.find("h1",{"class":"product-main__name"}).text.replace('\n',"")
    except:
        name=None

    whisky = {"name":name,"price":price,"rating":rating,"about":about}

    data.append(whisky)

In [16]:
# PRINT THE DATA EXTRACTED/SCRAPED FROM THE WEBSITE
df = pd.DataFrame(data)

print(df)

                                                  name    price  \
0                                Suntory Torys Classic   £30.45   
1                                           Nikka Days   £31.95   
2                                         Suntory Toki   £31.95   
3                                Akashi Blended Whisky   £32.75   
4                        Tokinoka White Blended Whisky   £32.95   
..                                                 ...      ...   
107             Karuizawa 30 Year OldSherry Cask #5347  £20,000   
108  Karuizawa 196742 Year Old Cask #6426 TWE 10th ...  £20,000   
109               Hanyu 1990Queen of Hearts Cask #9102  £25,000   
110                  Karuizawa 37 Year OldPearl Geisha  £35,000   
111               Hanyu 1985 The JokerMonochrome Label  £60,000   

              rating                                              about  
0     3.5(2 Reviews)  Suntory Torys Classic is a light and easy-drin...  
1        5(1 Review)  A vibrant addition to the

In [17]:
df.head()

,name,price,rating,about
0,Suntory Torys Classic,£30.45,3.5(2 Reviews),Suntory Torys Classic is a light and easy-drin...
1,Nikka Days,£31.95,5(1 Review),"A vibrant addition to the Nikka range, Nikka D..."
2,Suntory Toki,£31.95,4(30 Reviews),Toki is a blended whisky from Suntory's three ...
3,Akashi Blended Whisky,£32.75,3.5(19 Reviews),A lesser-known whisky outside the local Japane...
4,Tokinoka White Blended Whisky,£32.95,None,A Japanese blended whisky from White Oak disti...


In [18]:
#Hence its seen that i have scraped 112 products
df.shape

(112, 4)